In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.9/849.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.172
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!mkdir data
!wget -O ./data/anayasa.pdf https://www5.tbmm.gov.tr/anayasa/anayasa_2018.pdf

--2023-05-17 16:52:49--  https://www5.tbmm.gov.tr/anayasa/anayasa_2018.pdf
Resolving www5.tbmm.gov.tr (www5.tbmm.gov.tr)... 185.160.114.141
Connecting to www5.tbmm.gov.tr (www5.tbmm.gov.tr)|185.160.114.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 700913 (684K) [application/pdf]
Saving to: ‘./data/anayasa.pdf’

./data/anayasa.pdf  100%[===================>] 684.49K   591KB/s    in 1.2s    

2023-05-17 16:52:52 (591 KB/s) - ‘./data/anayasa.pdf’ saved [700913/700913]



In [35]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader

import os

In [34]:
os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY"

In [6]:
loader = PyPDFLoader("./data/anayasa.pdf")
pages = loader.load_and_split()

In [7]:
len(pages)

72

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
texts = text_splitter.split_documents(pages)

In [13]:
texts[0]

Document(page_content='1 \n TÜRKİYE CUMHURİYETİ ANAYASASI1 \nKanun No.: 2709 Kabul Tarihi: 7.11.1982 \nBAŞLANGIÇ  (Değişik: 23/7/1995-4121/1 md.) \nTürk Vatanı ve Milletinin ebedi varlığını ve Yüce Türk Devletinin bölünmez bütünlüğünü belirleyen bu \nAnayasa, Türkiye Cumhuriyetinin kurucusu, ölümsüz önder ve eşsiz kahraman Atatürk’ün belirlediği \nmilliyetçilik anlayışı ve O’nun inkılâp ve ilkeleri doğrultusunda;  \nDünya milletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin ebedi varlığı, \nrefahı, maddî ve manevî mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi yönünde;  \nMillet iradesinin mutlak üstünlüğü, egemenliğin kayıtsız şartsız Türk Milletine ait olduğu ve bunu millet \nadına kullanmaya yetkili kılınan hiçbir kişi ve kuruluşun, bu Anayasada gösterilen hürriyetçi demokrasi ve \nbunun icaplarıyla belirlenmiş hukuk düzeni dışına çıkamayacağı;  \nKuvvetler ayrımının, Devlet organları arasında üstünlük sıralaması anlamına gelmeyip, belli Devl

In [14]:
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [15]:
vectordb.persist()
vectordb = None

In [16]:
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [17]:
retriever = vectordb.as_retriever()

In [18]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [19]:
llm = OpenAI(temperature=0.7, model_name="text-davinci-003")

In [20]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [30]:
query = "Anayasa'nın 4. maddesi nedir?"
llm_response = qa_chain(query)
llm_response['result']

" Anayasa'nın 4. maddesi, Devletin şeklinin Cumhuriyet olduğu hakkındaki hüküm ile, 2. maddesindeki Cumhuriyetin nitelikleri ve 3. maddesi hükümlerinin değiştirilemez ve değiştirilmesi teklif edilemeyeceğini belirtir."

In [31]:
query = "Anayasa'nın 5. maddesi nedir?"
llm_response = qa_chain(query)
llm_response['result']

" Anayasa'nın 5. maddesi, Devletin temel amaç ve görevlerini tanımlar: Türk milletinin bağımsızlığını ve bütünlüğünü, ülkenin bölünmezliğini, Cumhuriyeti ve demokrasiyi korumak, kişilerin ve toplumun refah, huzur ve mutluluğunu sağlamak; kişinin temel hak ve hürriyetlerini, sosyal hukuk devleti ve adalet ilkeleriyle bağdaşmayacak surette sınırlayan siyasal, ekonomik ve sosyal engelleri kaldırmaya, insanın maddî ve manevî varlığının gelişmesi için gerekli şartları hazırlamaya çalışmaktır."

In [32]:
query = "Hangi madde devletin temel amaç ve görevlerini tanımlar?"
llm_response = qa_chain(query)
llm_response['result']

' Madde 5'

In [33]:
query = "Madde 6 nedir?"
llm_response = qa_chain(query)
llm_response['result']

' Madde 6, egemenliğin Millete ait olduğunu, Anayasanın koyduğu esaslara göre yetkili organların eliyle kullanılacağını ve hiçbir kişiye, zümreye veya sınıfa bırakılamayacağını belirtir.'

In [47]:
query = "Özel hayatımı kapsayan madde hangisidir?"
llm_response = qa_chain(query)
llm_response['result']

' Madde 20.'